Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Tutorial Digital Signal Processing

**STFT & Periodogram**,
Winter Semester 2023/24 (Course #24505)

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

Feel free to contact lecturer jacob.thoenes@uni-rostock.de

# Short-Time Fourier Transform (STFT) and Periodogram

## Motivation

The STFT and the inverse Short-Time Fourier Transform (iSTFT) are fundamental building blocks for digital signal processing (DSP).
With the STFT, the time-frequency varying characteristics of a signal can be analyzed.
Combining the STFT as the analyzing step and a subsequent iSTFT as the synthesizing/reconstruction step, builds the foundation for so called **time-frequency processing**, which is heavily used in nowadays DSP algorithms.

For certain algorithms the computational load can be considerably decreased using time-frequency processing.
Other approaches can only be realized due to explicit usage of time-frequency processing.
A very prominent example is the invention of [Auto-Tune](https://en.wikipedia.org/wiki/Auto-Tune) in the late 1990s. Furthermore, all video and audio codecs work this way.
IP-based streaming is only possible due to the transport of compressed time-frequency data rather than raw data. 

If we consider only the analysis stage, i.e. the STFT itself, we can elaborate a nice link to the periodogram: we can interpret this as a special post-processing of STFT data.

It thus appears reasonable that we have a detailed look to STFT/iSTFT and their key concept within this homework assignment.

## Objective

We aim at

- programming the Short-time Fourier Transform (STFT) by ourselves
- programming the Inverse Short-time Fourier Transform (iSTFT) by ourselves
    - using standard overlap-add (OLA)
    - using weigthed overlap-add (WOLA)
- normalize the STFT data either as a spectrum or as a power spectral density (PSD)
- make a surface plot of the magnitude of the STFT spectrum
- calculate PSDs from the STFT data
- apply the Welch method, i.e. averaging PSDs, to obtain a so called modified periodogram

In [ ]:
import matplotlib as mpl
import numpy as np
from matplotlib import pyplot as plt
from scipy.fft import fft, ifft
from scipy.signal import (
    check_COLA,
    check_NOLA,
    get_window,
    istft,
    spectrogram,
    stft,
    welch,
)

In [ ]:
if False:  # discrete colormap example with scatter plot
    x = np.arange(-6, 7)
    y = np.arange(-6, 7)
    z = np.arange(-6, 7) + 0

    cmap = plt.cm.viridis
    norm = mpl.colors.BoundaryNorm(np.arange(-6, 7, 1), cmap.N)
    fig, ax = plt.subplots(1, 1)
    c = ax.scatter(x, y, c=z, cmap=cmap, norm=norm, s=400, edgecolor="none")
    cbar = fig.colorbar(c, ax=ax, ticks=np.arange(-6, 7, 2), label="dB")
    plt.show()

In [ ]:
if False:  # check Parseval's theorem
    N = 2**10
    OmegaN = 2 * np.pi / N
    k = np.arange(0, N)
    y = np.cos(OmegaN * k)
    Y = fft(y)
    yvar = np.vdot(y, y) / N
    Yvar = (np.vdot(Y, Y) / N) / N
    ystd = np.sqrt(yvar)
    Ystd = np.sqrt(Yvar)
    print(yvar, ystd)
    print(Yvar, Ystd)

In [ ]:
def my_stft(x, fs, window, nperseg, noverlap, nfft):
    nhop = nperseg - noverlap
    X = np.zeros((nfft, 1))
    k = 0
    while k < len(x):
        tmp = x[k : k + nperseg]
        if len(tmp) == noverlap:  # works only due to power of two !!!
            # print('we do not include this last segment')
            break
        wtmp = window * tmp
        X = np.column_stack((X, np.array(fft(wtmp, nfft))))
        k += nhop
    X = X[:, 1:]
    # first time index is always nperseg//2 and then hopping according to:
    t = (np.arange(X.shape[1]) * (nperseg - noverlap) + nperseg // 2) / fs
    f = (2 * np.pi / X.shape[0] * np.arange(X.shape[0])) / (2 * np.pi) * fs
    return f, t, X


def my_istft(X, fs, window, nperseg, noverlap, nfft):
    nhop = nperseg - noverlap
    xola = np.zeros(len(x), dtype="complex")
    xwola = np.zeros(len(x), dtype="complex")
    xlms = np.zeros(len(x), dtype="complex")
    ola_sum = np.zeros(len(x), dtype="double")
    wola_sum = np.zeros(len(x), dtype="double")
    k = 0
    while k < X.shape[1]:
        xola[k * nhop : k * nhop + nperseg] += ifft(X[:, k]) * 1
        xwola[k * nhop : k * nhop + nperseg] += ifft(X[:, k]) * window
        xlms[k * nhop : k * nhop + nperseg] += ifft(X[:, k]) * window
        # check basic OLA constraint
        ola_sum[k * nhop : k * nhop + nperseg] += window
        wola_sum[k * nhop : k * nhop + nperseg] += window**2  # check WOLA constraint
        k += 1
    # check_imag = np.array([np.min(np.imag(xola)),  np.max(np.imag(xola)),
    #          np.min(np.imag(xwola)), np.max(np.imag(xwola)),
    #          np.min(np.imag(xlms)),  np.max(np.imag(xlms))])
    # print(np.max(check_imag))
    xola = np.real_if_close(xola)  # / np.max(ola_sum)
    xwola = np.real_if_close(xwola)  # / np.max(wola_sum)
    idx = wola_sum != 0  # if we have a zero somewhere, don't divide
    xlms[idx] /= wola_sum[idx]
    xlms = np.real_if_close(xlms)
    return xola, xwola, xlms, ola_sum, wola_sum


def my_norm_stft(X, w):
    # normalization wrt window type!!!, cf. Harris: Peak Power Gain
    return X / np.sqrt(w.sum() ** 2)


def my_norm_psd(X, w, fs):
    return (np.conjugate(X) * X) / (np.vdot(w, w) * fs)

# Introduction STFT / iSTFT

We should work through a brief summary of the STFT / iSTFT concepts before we start coding for the tasks.

## Variables
We define the following variables

- $-\infty \leq n\in\mathbb{Z} \leq +\infty$ is the sample index for whole time range
- $k\in\mathbb{Z}$ is --- as we usually define --- the sample index for a DFT block of length $N$, so $0\leq k \leq N-1$
- $N\in\mathbb{N}$ ist the DFT block size, here we use sizes corresponding to a power of two
- $\mu\in\mathbb{Z}$ is --- as we usually define --- the index for discrete DFT frequencies, $0\leq \mu \leq N-1$
- $\theta\in\mathbb{Z}$ is the index for discrete time blocks of the STFT, for ease of discussion we consider $n \geq 0$ and indicate the very first time block with $\theta=0$, thus $\theta\geq 0$; the mnemonic *theta* for *time* might help
- $H\in\mathbb{N}>0$ is the so called hop size, i.e. by how many samples we jump to get to the next block
- in software functions often the number of overlapped samples $N-H$ must be defined rather than the hop size

## Signal Flow Chart

Fig. 1 depicts the fundamental signal processing chain for time-frequency processing. We will have a detailed look at the STFT and iSTFT itself. So we need to discuss the analysis and the synthesis stage.
The theoretical foundations of the STFT analysis and synthesis were derived in the late 1970s and are referenced in the bibliography listing at the end of this document.
Keep this in mind when asking for detailed knowledge about STFT some time.
Particularly helpful might be chapter 7 in [Zoe11] as a convenient summary of key concepts. 

<img src="fig_72_zoelzer_redraw.png" width=600 />

**Fig. 1**: Adapted from Fig. 7.2 from [Zoe11]

## Analysis Stage

We consider the (theoretically infinite, by our above convention we start at $n=0$) discrete-time signal $x[n]$, which should be analyzed with the STFT approach. 

For that, we extend the Discrete Fourier Transform (DFT) $X[\mu]=\sum_{k=0}^{N-1}x[k]\cdot\mathrm{e}^{-\mathrm{j}\frac{2\pi}{N}k\mu}$ by a real-valued window $w[k]$ and by a temporal left shift (cf. chapter 10.3.4 in [OS10]) to

\begin{equation}
X[\mu,\theta] = \sum_{k=0}^{N-1} \underbrace{\left(x[\theta\cdot H+k] \, w[k]\right)}_{x_\theta} \cdot \mathrm{e}^{-\mathrm{j}\frac{2 \, \pi}{N} \, k \, \mu}.
\end{equation}

Recall our above  convention $\theta\geq 0$.

The notation of using finite length DFT blocks inherently implies 
$w[k] = 0\,\,\,\text{for}\,\,\,k<0\,\,\,\text{and}\,\,\,k>N-1$.
We will use windows that have none zeropadding parts over the length $N$.
We should follow the convention that in a **row** the **time varies**, and in a **column** the **frequency changes** when aligning $X[\mu,\theta]$ as two-dimensional, complex-valued data matrix $\mathbf{X}_{\mu,\theta}$.
Thus, we have access to complex-valued data points aligned over frequency and time indices.
This is the building block **STFT**, cf. the upper part of Fig. 1.

## Synthesis Stage

The **iSTFT** is then given from the inverse DFT of eq. (1) as

\begin{align}
\underbrace{x[\theta\cdot H + k] \, w[k]}_{x_\theta} =\frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}.
\end{align}

- Recall that the obtained time-domain signal blocks $x_\theta = x[\theta\cdot H+k] \cdot w[k]$ hold for $0\leq k\leq N-1$. Note that we calculate $x_q$ blocks rather than $x[\theta\cdot H+k]$ in the iSTFT stage. Proper alignment and superposition of all $x_q$ blocks then returns the signal $x[n]$ under certain conditions, which we will invent below as so called perfect reconstruction. 
- Absolute time index is $n=\theta\cdot H+k$.
- Blocks might overlap, when the hop size $H<N$.
- Note that it is unusual to set up $H>N$.
In such a case certain time ranges are completely missing and no perfect reconstruction can be achieved at all.
- The case $H=N$ is sometimes used for the Bartlett and Welch periodogram.

### Overlap-Add (OLA)

The reconstruction towards the signal $x[n]$ requires properly time-aligned overlapping and summation of iSTFT blocks.
This is depicted in the lower part of Fig. 1 and is known as **overlap-add** (OLA).
The OLA is already known from [segmented convolution](https://nbviewer.jupyter.org/github/spatialaudio/digital-signal-processing-lecture/blob/master/nonrecursive_filters/segmented_convolution.ipynb).
Here it is used exactly in the same manner: we have blocks aligned at different time instances, these blocks might overlap, and their superposition gives us the desired output signal.
We should have an initial expectation that an analysis/synthesis-pair for perfect reconstruction needs carefully chosen parameters, such as

- window characteristics
- window length / block size
- hop size
- signal pre-/post-processing steps

The STFT literature reports **three different** concepts on how to properly approach OLA:

1. the **basic OLA** (commonly abbreviated with OLA), this is the approach discussed with formulas above so far, considered in [Smi01], [OS10], [Rei15], in Matlab this is an option for the `istft()`

2. the **weighted OLA** (WOLA), this is what Fig. 1 actually shows, considered in [Smi01], [Zoe11], [Pul18], this is Matlab's default for `istft()`

3. the **least-squares error minimizing solution** (LSOLA), this is Python's default in `scipy.signal.istft()`

See [Gri84, eq. (6),(7),(8)] for a detailed elaboration.

In order to reinvent the ideas of these three concepts (the proofs of them are more sophisticated, but the following simple example works quite well), consider the trivial input signal $x[n]=1$ for all $n$.
The STFT and subsequent iSTFT yields the rectangular signal shifted to different time instances and windowed

\begin{align}
\underbrace{\mathrm{rect}_N[\theta\cdot H+k] \, w[k]}_{x_\theta} = \frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}.
\end{align}

#### Basic OLA

For OLA we need to sum these blocks as 
\begin{align}
\sum_\theta \mathrm{rect}_N[\theta\cdot H+k] \, w[k] =\sum_\theta  \frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}.
\end{align}
The question for aiming at perfect reconstruction is: Under which circumstances this equation becomes equal to $x[n] = 1$ ?!?

Recall, that the window is non-zero only for its arguments $[0...N-1]$.
Then, we can get rid of the (unit amplitude) rectangular function and deploy shifted windows directly.
This leads to the condition (absolute time index $n=\theta\cdot H+k$ rewritten as $k = n - \theta\cdot H$)
\begin{align}
\sum_\theta w[n -\theta\cdot H] = 1
\end{align}
for perfect reconstruction of our trivial example signal $x[n]=1$.
We can generalize this condition for any signal $x[n]$.
It simply means that all overlapping windows must sum to unity. Or, at least they must sum to $\sum_\theta w[n -\theta\cdot H] = \mathrm{const}$, by which the reconstructed signal can be normalized then.
This is known as the constant-overlap-add (**[COLA](https://ccrma.stanford.edu/~jos/sasp/Mathematical_Definition_STFT.html#19930)**) constraint for windows.

#### Weighted OLA

The WOLA adds **additional windowing after iSTFT**. For ease of discussion we assume this to be the same window as for the analysis stage. 
This yields

\begin{align}
w[k] \cdot \underbrace{\left(\mathrm{rect}_N[\theta\cdot H+k] \, w[k]\right)}_{x_\theta} = w[k] \cdot \left(\frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}\right),
\end{align}

and summing

\begin{align}
\sum_\theta \mathrm{rect}_N[\theta\cdot H+k] \, w^2[k] = \sum_\theta w[k] \cdot \left(\frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}\right).
\end{align}

This asks for the condition
\begin{align}
\sum_\theta w^2[n -\theta\cdot H] = 1
\end{align}

to reconstruct our trivial example signal $x[n]=1$.
We again can generalize this condition for any signal $x[n]$ to require $\sum_\theta w^2[n -\theta\cdot H] = \mathrm{const}$, known as the **[WOLA](https://ccrma.stanford.edu/~jos/sasp/Choice_WOLA_Window.html)** criterion.

#### Least Squares OLA

As an improvement of the WOLA for

- windows that not exactly fulfill the WOLA criterion and
- more generally, to find the least squares approximation of a desired STFT spectrum

[Gri84] showed that

\begin{align}
x[n] = \frac{\sum_\theta w[k] \cdot \mathrm{iDFT}\{X[\mu,\theta]\}}{\sum_\theta w^2[n -\theta\cdot H]}
\end{align}

yields the optimum solution.
So, instead of relying on the WOLA criterion (overlapping of squared windows must be constant), the reconstructed signal (numerator, WOLA approach) is divided by the window-overlap signal (denominator). However, this requires $\sum_\theta w^2[n -\theta\cdot H]\neq 0$ for any sample, which is known as nonzero overlap-add (**[NOLA](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.check_NOLA.html)**) constraint.
This approach is implemented in `scipy.signal.istft()`. This is convenient, since we don't have to care if WOLA criterion is met, but just need to check if NOLA holds.

In the first task section we want to implement our own STFT / iSTFT-pair for certain configurations.
These will then be investigated in terms of their COLA and WOLA constraints.

# Task 1: Generate Input Signal

Given:

- whole signal length $L=256$
- DFT block length $N = 64$

Generate a signal

\begin{equation}
x[n] = \cos(\Omega_N \, n) - \frac{1}{2}\sin(2\, \Omega_N \, n)
\end{equation}

in the time range $0 \leq n \leq L-1$ using the fundamental digital frequency $\Omega_N = \frac{2\pi}{N}$. Plot the signal over $n$ with proper axis labeling, x-ticks labeling with step size of 16 or 32 might be helpful.

In [ ]:
L = 256
N = 64
OmN = 2 * np.pi / N
n = np.arange(L)
x = np.cos(OmN * n) - 1 / 2 * np.sin(2 * OmN * n)
plt.plot(n, x)
plt.xlabel(r"$n$")
plt.ylabel(r"$x[n]$")
plt.xticks(np.arange(0, 256, 32))
plt.grid(True)

# Task 2: STFT

3P STFT, 1P H, 1P w, 1P dim X, freq, time

1. For the above defined finite signal $x[n]$ implement the STFTs (recall $\mu,\theta\in\mathbb{Z}$)

\begin{equation}
X[\mu,\theta] = \sum_{k=0}^{N-1} x[\theta \cdot H+k] \cdot w[k] \cdot \mathrm{e}^{-\mathrm{j}\frac{2 \, \pi}{N} \, k \, \mu}
\end{equation}

for **two different hop sizes**

- $H=16$
- $H=32$

and **two different windows**

- a normal, periodic (non-symmetric) Hann window, name the variable `w`

Matlab: `w = hann(64, 'periodic')`

Python: `w = get_window('hann', Nx=64, fftbins=True)` using `from scipy.signal import get_window`

- the square root of a normal, periodic (non-symmetric) Hann window, name the variable `ws`

Matlab: `ws = sqrt(hann(64, 'periodic'))`

Python: `ws = np.sqrt(get_window('hann', Nx=64, fftbins=True))`

2. Arrange complex-valued STFT data as 2D data matrices of form $\mathbf{X}_{\mu,\theta}$.

It is explicitly required to **implement the STFT by our own**.

For simplified programming, we might assume that $L/H\in\mathbb{N}$. Of course, the usage of the FFT (in Matlab: `fft()`, in Python: `numpy.fft.fft()` or `scipy.fft.fft()`) is allowed and meaningful.
We also might validate our STFT results against build-in functions.

So, in this subtask 2.2, we create four STFT matrices in total:

- variable `Xa` for $H=16$, normal Hann window
- variable `Xb` for $H=16$, square root Hann window
- variable `Xc` for $H=32$, normal Hann window
- variable `Xd` for $H=32$, square root Hann window


3. What are the dimensions of the STFT matrices?

4. We assume a sampling frequency $f_s=4$ Hz. For that state the physical time and frequency vectors, i.e. all values of $\mu \Delta f$ and $\theta \Delta t$. Recall that $\mu$ and $\theta$ start with zero by  our convention.


In [ ]:
# STFT
fs = 4  # Hz
nperseg = N
w = get_window("hann", Nx=nperseg, fftbins=True)

noverlap = 3 * nperseg // 4
nhop = nperseg - noverlap
f, t, Xa = my_stft(x, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg)
f, t, Xb = my_stft(
    x, fs=fs, window=np.sqrt(w), nperseg=nperseg, noverlap=noverlap, nfft=nperseg
)
print("N = ", N, ", H = ", nhop)
print("dim X:", Xa.shape)
print("f = ", f, " Hz")
print("t = ", t, " s")

noverlap = nperseg // 2
nhop = nperseg - noverlap
f, t, Xc = my_stft(x, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg)
f, t, Xd = my_stft(
    x, fs=fs, window=np.sqrt(w), nperseg=nperseg, noverlap=noverlap, nfft=nperseg
)
print("N = ", N, ", H = ", nhop)
print("dim X:", Xc.shape)
print("f = ", f, " Hz")
print("t = ", t, " s")

Note: We might consider to implement tasks 3 to 6 as a single block.
Note that certain functionalities are repeatedly required, which could be conveniently encapsulated in functions.

# Task 3: Inverse STFT

Implement own written source code that calculates the inverse STFT from the matrix $\mathbf{X}_{\mu,\theta}$, i.e. all possible blocks $x_\theta$ according to

\begin{align}
\underbrace{x[\theta\cdot H+k] \cdot w[k]}_{x_\theta}= \frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}
\end{align}
for $0\leq k \leq N-1$ and all required $\theta$.

# Task 4: Overlap-Add Reconstruction

Implement own written source code to reconstruct the signal by basic **overlap add** (OLA) for the acquired STFTs 

- `Xa` for $H=16$, normal Hann window, call the result variable `xa`

- `Xc` for $H=32$, normal Hann window, call the result variable `xc`

by

\begin{align}
\sum_{\theta} \underbrace{x[\theta\cdot H+k] \cdot w[k]}_{x_\theta}= \sum_{\theta} \frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}.
\end{align}

# Task 5: Weighted Overlap Add Reconstruction

Implement own written source code to reconstruct the signal by **weighted overlap add** (WOLA) for the acquired STFTs 

- `Xb` for $H=16$, square root Hann window, call the variable `xb`

- `Xd` for $H=32$, square root Hann window, call the variable `xd`

by

\begin{align}
\sum_{\theta} \left(\underbrace{\left(x[\theta\cdot H+k] \cdot w[k]\right)}_{x_\theta} \cdot w[k] \right)= \sum_{\theta} \left(\left(\frac{1}{N}\sum_{\mu=0}^{N-1}X[\mu,\theta]\cdot\mathrm{e}^{+\mathrm{j}\frac{2\pi}{N}k\mu}\right) \cdot w[k]\right).
\end{align}

# Task 6: Check COLA Conditions

Implement own written source code to check the COLA / WOLA criteria

\begin{align}
c_\mathrm{OLA}[n] = \sum_\theta w[n -\theta\cdot H] = \mathrm{const},
\end{align}

\begin{align}
c_\mathrm{WOLA}[n] = \sum_\theta w^2[n -\theta\cdot H] = \mathrm{const}
\end{align}

on the used windows for all four cases. So, we have

- $c_\mathrm{OLA}$ and $c_\mathrm{WOLA}$ for case `xa`
- $c_\mathrm{OLA}$ and $c_\mathrm{WOLA}$ for case `xb`
- $c_\mathrm{OLA}$ and $c_\mathrm{WOLA}$ for case `xc`
- $c_\mathrm{OLA}$ and $c_\mathrm{WOLA}$ for case `xd`.

In [ ]:
# iSTFT
noverlap = 3 * nperseg // 4
nhop = nperseg - noverlap
print(nhop)
xa, _, _, ola_suma, wola_suma = my_istft(
    Xa, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg
)
_, xb, _, ola_sumb, wola_sumb = my_istft(
    Xb, fs=fs, window=np.sqrt(w), nperseg=nperseg, noverlap=noverlap, nfft=nperseg
)
noverlap = nperseg // 2
nhop = nperseg - noverlap
print(nhop)
xc, _, _, ola_sumc, wola_sumc = my_istft(
    Xc, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg
)
_, xd, _, ola_sumd, wola_sumd = my_istft(
    Xd, fs=fs, window=np.sqrt(w), nperseg=nperseg, noverlap=noverlap, nfft=nperseg
)

plt.plot(ola_suma, label="ola_suma")
plt.plot(ola_sumb, label="ola_sumb")
plt.plot(ola_sumc, label="ola_sumb")
plt.plot(ola_sumd, label="ola_sumb")
plt.legend()
plt.grid(True)

In [ ]:
plt.plot(wola_suma, label="wola_suma")
plt.plot(wola_sumb, label="wola_sumb")
plt.plot(wola_sumc, label="wola_sumb")
plt.plot(wola_sumd, label="wola_sumb")
plt.legend()
plt.grid(True)

In [ ]:
plt.subplot(211)
plt.subplots_adjust(hspace=0.5)
# plot Xa
plt.title("selfmade iSTFT for Xa")
plt.plot(
    xb
)  # plot the result, the time-base has to be multiplied by 4 because of the sampling frequency
plt.plot(x)  # plot the input value
plt.legend(["ISTFT", "input"])

plt.title("selfmade iSTFT for Xa")
plt.plot(
    xd
)  # plot the result, the time-base has to be multiplied by 4 because of the sampling frequency
plt.plot(x)  # plot the input value
plt.legend(["ISTFT", "input"])

# Task 7: Plot Signals

We created some signals which we now should plot to inspect and validate.
Also, feel free to use plots of intermediate results as well.
See the figure with the four subplots visualizing the four cases.

We shall generate these plots on our own. Each subplot shall contain

- input signal $x[n]$
- window $w[n]$
- shifted window by one hop size
- the cola signals $c_\mathrm{OLA}[n]$ and $c_\mathrm{WOLA}[n]$
- the error signal $x_\mathrm{reconstructed}[n] - x[n]$, e.g. `xa-x`

Pay attention to properly given legend, title, axis ticks and labeling, as well as meaningful plotting ranges.

In [ ]:
plt.figure(figsize=(10, 7))
plt.subplot(2, 2, 1)
noverlap = 3 * nperseg // 4
nhop = nperseg - noverlap
plt.plot(x, label="x", color="C2")
plt.plot(w, label="w", color="k")
plt.plot(np.arange(0, nperseg) + nhop, w, color="dimgray")
plt.plot(ola_suma, "-", label=r"$c_{OLA}$", color="C0")
plt.plot(wola_suma, "-", label=r"$c_{WOLA}$", color="C3")
plt.plot(xa - x, "-.", label="x$_{a}$-x", color="C1")
plt.xticks(np.arange(0, len(x) + nperseg, nperseg // 2))
plt.xlim(0, L)
plt.ylim(-2, 3)
plt.legend()
plt.ylabel("")
plt.title("a) OLA, N$_{DFT}$=" + str(nperseg) + ", H$_{hop}$=" + str(nhop))
plt.grid(True)

plt.subplot(2, 2, 2)
noverlap = 3 * nperseg // 4
nhop = nperseg - noverlap
plt.plot(x, label="x", color="C2")
plt.plot(np.sqrt(w), label="ws", color="k")
plt.plot(np.arange(0, nperseg) + nhop, np.sqrt(w), color="dimgray")
plt.plot(ola_sumb, "-", label=r"$c_{OLA}$", color="C0")
plt.plot(wola_sumb, "-", label=r"$c_{WOLA}$", color="C3")
plt.plot(xb - x, "--", label="x$_{b}$-x", color="C1")
plt.xticks(np.arange(0, len(x) + nperseg, nperseg // 2))
plt.xlim(0, L)
plt.ylim(-2, 3)
plt.legend()
plt.ylabel("")
plt.title("b) WOLA, N$_{DFT}$=" + str(nperseg) + ", H$_{hop}$=" + str(nhop))
plt.grid(True)

plt.subplot(2, 2, 3)
noverlap = nperseg // 2
nhop = nperseg - noverlap
plt.plot(x, label="x", color="C2")
plt.plot(w, label="w", color="k")
plt.plot(np.arange(0, nperseg) + nhop, w, color="dimgray")
plt.plot(ola_sumc, "-", label=r"$c_{OLA}$", color="C0")
plt.plot(wola_sumc, "-", label=r"$c_{WOLA}$", color="C3")
plt.plot(xc - x, "-.", label="x$_{c}$-x", color="C1")
plt.xticks(np.arange(0, len(x) + nperseg, nperseg // 2))
plt.xlim(0, L)
plt.ylim(-2, 3)
plt.legend()
plt.xlabel("n")
plt.ylabel("")
plt.title("c) OLA, N$_{DFT}$=" + str(nperseg) + ", H$_{hop}$=" + str(nhop))
plt.grid(True)

plt.subplot(2, 2, 4)
noverlap = nperseg // 2
nhop = nperseg - noverlap
plt.plot(x, label="x", color="C2")
plt.plot(np.sqrt(w), label="ws", color="k")
plt.plot(np.arange(0, nperseg) + nhop, np.sqrt(w), color="dimgray")
plt.plot(ola_sumd, "-", label=r"$c_{OLA}$", color="C0")
plt.plot(wola_sumd, "-", label=r"$c_{WOLA}$", color="C3")
plt.plot(xd - x, "--", label="x$_{d}$-x", color="C1")
plt.xticks(np.arange(0, len(x) + nperseg, nperseg // 2))
plt.xlim(0, L)
plt.ylim(-2, 3)
plt.legend()
plt.xlabel("n")
plt.ylabel("")
plt.title("d) WOLA, N$_{DFT}$=" + str(nperseg) + ", H$_{hop}$=" + str(nhop))
plt.grid(True)

# Task 8: Interpretation

Graphical visualization always requires interpretation.

One observation would be the fade in and fade out at the beginning and the end of the considered finite length.

- Why is the case?

Let us ignore these fading artifacts, then give an explained answer to the following questions:

- In which cases a,b,c,d the COLA constraint (signal $c_\mathrm{OLA}[n]$) indicates that reconstruction is feasible?
- In which cases a,b,c,d the WOLA constraint (signal $c_\mathrm{WOLA}[n]$) indicates that reconstruction is feasible?
- In which cases a,b,c,d perfect reconstruction is achieved **without** any further post-processing?
- In which cases a,b,c,d perfect reconstruction is achieved **only with** further post-processing?
- Name possible post-processing algorithms.
- Which other windows could have been used to obtain similar results. Why?

# What we got so far...

By now, we have implemented the basic structure of time-frequency processing and hopefully got some detailed insights of the data structure of an STFT and the different windowing concepts.

Note, that our above analysis and synthesis/reconstruction assumes **unmodified STFT data**.

For time-frequency processing, certain modifications in magnitude and phase (such as filtering) are vivid parts of the DSP job. Then the signal processing needs some **further processing steps**. These can be seen in Fig. 2 and comprise of

- additional zero-phase shift, achieved by a circular shift of the time block
- additional phase shift, such that the phase in each STFT frame refers to the original starting time of the input signal. This is especially required if we want to modify the phase

before time-frequency processing, and afterwards the reversed operations.
Furthermore, to avoid time-aliasing often the blocks are zeropadded prior to the FFT. This is related to [linear convolution using cyclic convolution](https://nbviewer.jupyter.org/github/spatialaudio/digital-signal-processing-lecture/blob/master/nonrecursive_filters/fast_convolution.ipynb).
Please, be aware of this and elaborate on these facts once required sometime.

<img src="fig_75_zoelzer_redraw.png" width=450 />

**Fig. 2**: Adapted from Fig. 7.5 from [Zoe11]

# Reference Implementation 2: STFT / iSTFT

check with Python

In [ ]:
L = 256
N = 64
OmN = 2 * np.pi / N
n = np.arange(L)
x = np.cos(OmN * n) - 1 / 2 * np.sin(2 * OmN * n)

fs = 4
nfftM = 1
nperseg = N
# noverlap = 3*nperseg//4
noverlap = nperseg // 2
nhop = nperseg - noverlap
w = get_window("hann", Nx=nperseg, fftbins=True)
print("nperseg:", nperseg, ", noverlap:", noverlap, ", hop size:", nhop)
print("COLA condition:", check_COLA(w, nperseg, noverlap, tol=1e-15))
print("NOLA condition:", check_NOLA(w, nperseg, noverlap, tol=1e-15))
if False:
    w = w  # Basic OLA
    stft_type = "basic OLA"
else:
    w = np.sqrt(w)  # WOLA
    stft_type = "weighted OLA"
print(stft_type)

# my STFT
f, t, X = my_stft(
    x, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg * nfftM
)
# scipy STFT
ftmp, ttmp, Xtmp = stft(
    x,
    fs=fs,
    window=w,
    nperseg=nperseg,
    noverlap=noverlap,
    nfft=nperseg * nfftM,
    detrend=False,
    return_onesided=False,
    boundary=None,
    padded=False,
    axis=-1,
)
Xtmp *= np.sum(w)
print(
    "Compare STFTs my vs. signal:\nf equal:",
    np.allclose(f[1 : len(f) // 2], ftmp[1 : len(ftmp) // 2]),
    ", t equal:",
    np.allclose(t, ttmp),
    ", X equal:",
    np.allclose(X, Xtmp),
)
print("dim X:", X.shape)
print("f = ", f, " Hz")
print("t = ", t, " s")

In [ ]:
# scipy iSTFT
ttmp, xtmp = istft(
    X,
    fs=fs,
    window=w,
    nperseg=nperseg,
    noverlap=noverlap,
    nfft=nperseg * nfftM,
    input_onesided=False,
    boundary=None,
    time_axis=-1,
    freq_axis=-2,
)
xtmp /= np.sum(w)
xtmp = np.real_if_close(xtmp)

print(X.shape, xtmp.shape)

# my iSTFT
xola, xwola, xlms, ola_sum, wola_sum = my_istft(
    X, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg * nfftM
)

print("Compare iSTFTs my vs. signal:", np.allclose(xtmp, xlms))

plt.plot(x, label="x", color="C2")
plt.plot(w, label="w", color="k")
plt.plot(np.arange(0, nperseg) + nhop, w, color="dimgray")
plt.plot(ola_sum, "-", label=r"$\Sigma w_{OLA}$", color="C0")
plt.plot(wola_sum, "-", label=r"$\Sigma w_{WOLA}$", color="C3")
plt.plot(xola - x, "-.", label="x$_{OLA}$-x", color="C0")
plt.plot(xwola - x, "--", label="x$_{WOLA}$-x", color="C3")
plt.plot(xlms - x, ":", label="x$_{LMS}$-x", color="C1")
plt.xticks(np.arange(0, len(x) + nperseg, nperseg // 2))
plt.xlim(0, L)
plt.ylim(-2, 3)
plt.legend()
plt.xlabel("n")
plt.ylabel("")
plt.title(
    stft_type
    + ", nperseg="
    + str(nperseg)
    + ", noverlap="
    + str(noverlap)
    + ", nhop="
    + str(nhop)
)
plt.grid(True)

# STFT Spectrum

As a second task section, we should investigate the magnitude spectrum of an STFT as a surface plot, i.e. colored magnitude over time and frequency axis in one graph. We approach this by programing our own example using our own STFT. Once we've mastered that, we can have a closer look to other examples on our own.

# Task 9: Generate Input Signal

For $L = 2048$, $M=128$, $N = 64$ consider the following discrete-time signal

\begin{equation}
x[n] = \sum_{\mu=8}^{23} \sum_{n=0}^{L-1} \mathrm{rect}_{M}[n-M (\mu-8)] \cdot \cos(\frac{2\pi}{N} \mu n) \cdot 2 \, \sqrt{2}.
\end{equation}

Recall that
\begin{align}
\mathrm{rect}_M[n] =
\begin{cases}
1 \quad \mathrm{for} \quad 0\leq n \leq M-1\\
0 \quad \mathrm{else}.
\end{cases}
\end{align}

- Write own source code to generate the signal, store it in variable `x`.

Before actual programming, we should ask what this signal actually consists of (this is a typical and of course intended example, where the analytic equation looks more complicated than a concise implementation in software, in Python this probably could be done with a one-liner). The following thoughts might help:

- What do $L$, $M$ and $N$ actually do?
- Why do we use $\mu$ and $n$ variables just as in a DFT?
- What does the rect function actually do?

We should not forget to implement the $2 \, \sqrt{2}$ factor.

- State the [unbiased estimates](https://en.wikipedia.org/wiki/Bias_of_an_estimator) of the mean, the standard deviation and variance for `x`.

In [ ]:
fs = 1
N = 2**6
L = 2**11
OmegaN = 2 * np.pi / N
x = np.zeros(L)
for mu in range(8, 24):
    # print((mu-8)*2*N,(mu-8)*2*N+2*N-1)
    k = np.arange((mu - 8) * 2 * N, (mu - 8) * 2 * N + 2 * N)
    x[k] += np.cos(OmegaN * k * mu) * 2 * np.sqrt(2)
print("len(x):", len(x), "-> 2**", np.log(len(x)) / np.log(2))

print("mean:", np.mean(x), "\nstd:", np.std(x, ddof=1), "\nvar:", np.var(x, ddof=1))
xrms = np.sqrt((np.vdot(x, x) / len(x)))
print("xRMS:", xrms)

plt.plot(x)

# Task 10: Generate Gaussian-White Noise

- For the same signal length as $x[n]$ / `x` generate a random noise signal drawn from a normal distribution with zero mean and standard deviation $\sigma = \frac{3}{2}$. Store the data in signal vector `noise`.
- State the unbiased estimates of the mean, the standard deviation and the variance for `noise`.  

If we use `numpy` we might set up `np.random.seed(seed=1)` to obtain same results as referenced below. 

In [ ]:
sigma = 3 / 2
np.random.seed(seed=1)  # get always the same results for debug
noise = sigma * np.random.randn(
    L,
)  # noise
print(
    "mean:",
    np.mean(noise),
    "\nstd:",
    np.std(noise, ddof=1),
    "\nvar:",
    np.var(noise, ddof=1),
)
nrms = np.sqrt((np.vdot(noise, noise) / len(noise)))
print("xRMS:", nrms)

# Task 11: Mixing Input Signal with Noise Signal

- Add `x` and `noise` and store the result as `xn`.
- What are the theoretical values for the resulting mean, standard deviation and variance of this signal mixture?
- State the unbiased estimates of the mean, the standard deviation  and variance for `xn`. Do they fulfill our expectation?

In [ ]:
xn = 1 * x + 1 * noise
print("mean:", np.mean(xn), "\nstd:", np.std(xn, ddof=1), "\nvar:", np.var(xn, ddof=1))
xnrms = np.sqrt((np.vdot(xn, xn) / len(xn)))
print("theory std:", np.sqrt(2**2 + (3 / 2) ** 2))
print("theory var:", (2**2 + (3 / 2) ** 2))

plt.plot(xn)

# Task 12: STFT Magnitude Spectrum

For the created signal mixture `xn` calculate the STFT matrix $\mathbf{X}_{\mu,\theta}$ for

- DFT length $N=64$ samples
- hop size $H=32$ samples
- normal, periodic (non-symmetric) Hamming window (**not the Hann window as above**)

We should use our own STFT code from above to solve this task.
However, here we are free to use other STFT implementations, but then we need to take care of proper normalization.

- Store the STFT matrix in variable `XN`.

For **our** implementation, we normalize by the squared peak power gain of the utilized window $w$. Furthermore we calculate the magnitude of the complex data.
This reads
\begin{align}
\frac{|\mathbf{X}_{\mu,\theta}|}{\sqrt{\left(\sum_{k=0}^{N-1} w[k]\right)^2}}
\end{align}

which should be stored into `XNMag`.
Note that $\left(\sum_{k=0}^{N-1} w[k]\right)^2 = W^2(\mathrm{e}^{\mathrm{j}\Omega=0})$ is the so called peak power gain, cf. equation (10) in [Har78].
The maximum value of 20 lg(`XNMag`) should be very close to 4.74 dB. The exact value depends on the random noise `noise`. 
Furthermore, we might validate that max(abs(`XNMag`))$\approx\sqrt{2}$ when the amplitude of the **noise signal** is set to **zero**. 

- The value $\sqrt{2}$ results not by accident. Explain why.

In [ ]:
nperseg = 2**6
noverlap = 2**5
nhop = nperseg - noverlap
print("nperseg:", nperseg, ", noverlap:", noverlap, ", hop size:", nhop)

w = get_window("rect", nperseg)
w = get_window("hamming", nperseg)

f, t, XN = my_stft(
    xn, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg * nfftM
)
XN = my_norm_stft(XN, w)
#
f1, t1, X1 = stft(
    xn,
    fs=fs,
    window=w,
    nperseg=nperseg,
    noverlap=noverlap,
    nfft=nperseg,
    detrend=False,
    return_onesided=False,
    boundary=None,
    padded=False,
    axis=-1,
)
#
f2, t2, X2 = spectrogram(
    xn,
    fs=fs,
    window=w,
    nperseg=nperseg,
    noverlap=noverlap,
    nfft=nperseg,
    detrend=False,
    return_onesided=False,
    scaling="spectrum",
    axis=-1,
    mode="complex",
)
print("check scipy vs. my:")
print(np.allclose(X1, X2), np.allclose(f1, f2), np.allclose(t1, t2))
print(
    np.allclose(XN, X2),
    np.allclose(f[1 : len(f) // 2], f2[1 : len(f2) // 2]),
    np.allclose(t, t2),
)
print(
    np.allclose(XN, X1),
    np.allclose(f[1 : len(f) // 2], f1[1 : len(f1) // 2]),
    np.allclose(t, t1),
)

print("XN:", XN.shape)  # r...f, c...t
print("f:", f.shape)
print("t:", t.shape)

In [ ]:
XNmag = np.abs(XN)
XNdb = 20 * np.log10(XNmag)
print(np.max(XNmag))
print(np.max(XNdb))

# Task 13: Surface Plot of STFT Spectrum

See the figure below with the two subplots visualizing the STFT spectrum as level (in dB, left) and as magnitude (right).

- We shall generate these plots on our own from the data `XNmag`.

The surface plot might be realized with Matlab's `surf()` with `FaceColor='flat'` and `EdgeColor='none'` or 
Matplotlib's / PyPlot's `pcolormesh()` with `shading='flat'` and `edgecolors='face'`. 
Note, that for plotting no further manual data post processing is required.
Only the correct time and frequency vectors must be set up for **flat shading**.
This requires that the plot starts at time and frequency values of zero (see the pyplot/Matlab help files how flat shading aligns the coordinates with the color patches). 

Pay special attention that the colorbar has a discretized colormap.
Have a look to a matplotlib [surface plot example](https://github.com/spatialaudio/digital-signal-processing-exercises/blob/master/jupyter/jupyter_intro.ipynb) on how to do this.
Suitable colormaps are Matlab's new standard `parula` and matplotlib's `viridis`, `cividis`, `inferno`, `plasma` and `magma`. The figures below use the `inferno` colormap (weird naming?!). Please do not use `jet` (Matlab's earlier default) and `hsv`, since these colormaps do not consider that we have different perceptual sensitivity for different colors.

- Interpret these plots.

In [ ]:
t = np.insert(t, 0, 0.0)
f = np.insert(
    f, 0, 0.0
)  # needed for flat shading, https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pcolormesh.html

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(14 // 1.25, 6 // 1.25)

axt = ax[0]
cmap = plt.cm.inferno  # viridis #cividis #inferno #plasma #magma
dbmin = -24
dbmax = 9
dbstep = 3
norm = mpl.colors.BoundaryNorm(np.arange(dbmin, dbmax + dbstep, dbstep), cmap.N)

c = axt.pcolormesh(
    t * fs, f / fs, XNdb, cmap=cmap, norm=norm, shading="flat", edgecolors="face"
)
cbar = fig.colorbar(c, ax=axt, ticks=np.arange(dbmin, dbmax + dbstep, 3), label="dB")
axt.set_xlim(0, len(x))
axt.set_xticks(np.arange(0, len(x) + 2**8, 2**8))
axt.set_ylim(0, 1)
axt.set_yticks(np.arange(0, 1.25, 0.25))
axt.set_title("STFT Level")
axt.set_ylabel(r"frequency $\Omega \, / \, 2\pi$")
axt.set_xlabel("time index n")

axt = ax[1]
cmap = plt.cm.inferno
dbmin = 0
dbmax = 2
dbstep = 0.1
norm = mpl.colors.BoundaryNorm(np.arange(dbmin, dbmax + dbstep, dbstep), cmap.N)
c = axt.pcolormesh(
    t * fs, f / fs, XNmag, cmap=cmap, norm=norm, shading="flat", edgecolors="face"
)
cbar = fig.colorbar(c, ax=axt, ticks=np.arange(dbmin, dbmax + dbstep, 0.2), label="lin")
axt.set_xlim(0, len(x))
axt.set_xticks(np.arange(0, len(x) + 2**8, 2**8))
axt.set_ylim(0, 1)
axt.set_yticks(np.arange(0, 1.25, 0.25))
axt.set_title("STFT Magnitude")
axt.set_ylabel(r"frequency $\Omega \, / \, 2\pi$")
axt.set_xlabel("time index n");

In [ ]:
if False:
    # TBD:
    print(nfftM)
    print("xnrms:", xnrms)
    print("std of time slices:")
    XNstd = np.zeros(XN.shape[1])
    for c in range(XN.shape[1]):
        XNstd[c] = np.real(np.sqrt(np.vdot(XN[:, c], XN[:, c])) / np.sqrt(nfftM))
    print(XNstd)
    print("mean=", np.mean(XNstd), ", std=", np.std(XNstd))
    # aus diesen Werten unbiased Schätzer für Mittelwert, Median, Std

By now, we got an idea, how to approach an analysis of time-frequency varying signal characteristics with the above surface plot.

In other analysis tasks we are interested in the power spectral density of a signal. We will approach this with the third task section in the following.

# Introduction to the Periodogram

## Power Spectral Density Estimator

From the DSP lecture we know that the power spectral density (PSD) $\hat{\Phi}_{xx}(\Omega)$ can be estimated by 

\begin{equation}
\hat{\Phi}_{xx}(\Omega) = \frac{1}{N \cdot U} |X(\Omega)|^2,
\end{equation}

where the DTFT $X(\Omega)$ might be approximated with a (very large) $N$-block DFT

\begin{equation}
X(\Omega) \approx \sum_{k=0}^{N-1} w[k] x[k] \mathrm{e}^{-\mathrm{j}\frac{2\pi}{N} k \mu}.
\end{equation}

for input signal $x[k]$ and window $w[k]$. The normalization constant $U$ is given by

\begin{align}
U = \frac{f_s}{N} \sum_{k=0}^{N-1} w^2[k]
\end{align}

for a given sampling frequency $f_s$ in Hz.
**Note**: Set $f_s= 2\pi$ if PSD should be evaluated for digital frequency domain $\Omega$ (which we will do below!).
Thus, we recognize the DFT / DTFT frequency resolution $\Delta f = \frac{f_s}{N}$ / $\Delta \Omega = \frac{2 \pi}{N}$ in the constant $U$.
After canceling $N$, the PSD estimator becomes

\begin{equation}
\hat{\Phi}_{xx}(\mathrm{e}^{\mathrm{j}\Omega}) = \frac{|X(\Omega)|^2}{f_s \cdot \sum_{k=0}^{N-1} w^2[k]}.
\end{equation}

It has physical dimension of V$^2$ / Hz if $x(t)$ has dimension of V for physical sampling frequency in Hz.
If dealing in digital domain, the unit of the PSD and the averaged PSD is (signal amplitude)$^2$ / (rad/s).

## Periodogram

If we derive the PSD estimator from DFT data and for a rectangular window, the so called periodogram is obtained. For very large $N$ this estimator is unbiased. However, it is **not** a **[consistent](https://en.wikipedia.org/wiki/Consistent_estimator)** estimate (cf. the [lecture](https://github.com/spatialaudio/digital-signal-processing-lecture/blob/master/spectral_estimation_random_signals/introduction.ipynb)).

## Periodogram Averaging

The periodogram exhibits more or less (but never zero) deviations from the true PSD.
A naive approach would be taking an average of multiple periodograms in order to check what signal characteristics 'remains steady'.
That's what we would intuitively do in practice, right?
Of course there is a strong theoretical foundation and statistical proof for this approach, see e.g. chapter 10.5 in [OS10].
By taking the mean over $K$ periodograms

\begin{equation}
\frac{1}{K}\sum_K \hat{\Phi}_{xx}(\Omega) = \frac{1}{K}\sum_K \frac{|X(\Omega)|^2}{f_s \cdot \sum_{k=0}^{N-1} w^2[k]}.
\end{equation}

we obtain an **unbiased** AND **consistent** estimate of the real PSD (cf. the [lecture](https://github.com/spatialaudio/digital-signal-processing-lecture/blob/master/spectral_estimation_random_signals/introduction.ipynb)).
This averaging is often referred to as the **modified periodogram**.

### Bartlett's Method

The Bartlett method considers non-overlapped, rectangular windowed signal blocks for periodogram averaging.

### Welch's Method

The Welch method considers overlapped blocks using a **non-rectangular** window.
That sounds nice, since we could use our elaborated STFT stuff from above :-)

Thus, in the next task, we are going to implement the Welch method on our own.

# Task 14: Welch Periodogram

Here the task is to calculate the **Welch** peridogram for the STFT data `XN` created in task 12. So step by step we proceed as follows: 

1. We use the same parameters as for the surface plot

 - DFT length $N=64$ samples
 - hop size $H=32$ samples
 - normal, periodic (non-symmetric) Hamming window

to create STFT matrix `XN` of the input signal `xn`. 

2. The above discussed normalization to `XN` needs to be performed.

3. The above discussed mean, i.e. $\frac{1}{K} \sum_K \dots$ is calculated. This yields the Welch periodogram.

We assume a sampling frequency of $2 \pi$. This will give us the PSD estimate over **digital angular frequency** $\Omega$.
Of course, we have only data at discrete DFT frequencies, but this is ok for our purpose.
Though, we could interpolate DTFT data from DFT data, see [exercise](https://github.com/spatialaudio/digital-signal-processing-exercises/blob/master/dft/dft_to_dtft_interpolation.ipynb).

4. Store the Welch periodogram in variable `PxxWelch` and the angular frequencies of the DFT in `Om`.

In [ ]:
nperseg = 2**6
noverlap = 2**5
nhop = nperseg - noverlap
print("nperseg:", nperseg, ", noverlap:", noverlap, ", hop size:", nhop)

# w = get_window('rect', nperseg)
w = get_window("hamming", nperseg)

# !!! this is for digital anguar frequency-> PSD is of unit amplitude^2 / (rad/s)
fs = 2 * np.pi
f1, t1, X1 = spectrogram(
    xn,
    fs=fs,
    window=w,
    nperseg=nperseg,
    noverlap=noverlap,
    nfft=nperseg * nfftM,
    detrend=False,
    return_onesided=False,
    scaling="density",
    axis=-1,
    mode="psd",
)
f, t, X = my_stft(
    xn, fs=fs, window=w, nperseg=nperseg, noverlap=noverlap, nfft=nperseg * nfftM
)
X = my_norm_psd(X, w, fs)  # apply psd normalization
print("check scipy vs. my:")
print(np.allclose(X, X1))
print("X:", X.shape)  # r...f, c...t
print("f:", f.shape)
print("t:", t.shape)

#  Task 15: Plot the Welch Periodogram

See the figure below showing an exemplary Welch periodogram of `xn`.
This might slightly vary for different runtimes, when using a random noise signal.
As validation we might check that the maximum in this plot is about 2.5868, when **noise signal** amplitude is set to **zero**. By the way: is the plot for this case as we expect it?

1. Generate the plot.
2. What signal post-processing on `PxxWelch` is additionally required to obtain the plot? See hint below.
3. Discuss the plotted result.
4. Check that the variance of `xn` is the same as the cumulated power of the plotted PSD.

**Hint**:

The DFT frequency range is $0 \leq \mu \leq N-1$ for our STFT / Welch periodogram data.
However, we want to plot only the physical part of the spectrum in terms of correct magnitude, so we need to plot from $0 \leq \mu \leq \frac{N}{2}$ (this holds for N even!).
Thus, there is further (simple) post-processing required on the data. We treated this in one of the DFT exercises.

In [ ]:
X_my_welch = np.real(np.mean(X, axis=1))  # the Welch approach
X_my_welch[0] = X_my_welch[0] / 2  # DC treatment
X_my_welch[len(X_my_welch) // 2] = (
    X_my_welch[len(X_my_welch) // 2] / 2
)  # fs/2 treatment
X_my_welch_CorrectMag = 2 * X_my_welch

Om, Xw = welch(
    xn,
    fs=2 * np.pi,
    window=w,
    nperseg=nperseg,
    noverlap=noverlap,
    nfft=nperseg,
    detrend=False,
    return_onesided=True,
    scaling="density",
    axis=-1,
)

print(
    "Pxx vs. scipy Pxx:", np.allclose(Xw, X_my_welch_CorrectMag[0 : nperseg // 2 + 1])
)
print("signal var:", np.var(xn))
print("complete power in scipy Pxx=", np.sum(Xw) * 2 * np.pi / nperseg)
print(
    "complete power in my Pxx=",
    np.sum(X_my_welch_CorrectMag[0 : nperseg // 2 + 1]) * 2 * np.pi / nperseg,
)
print(np.max(Xw))


plt.plot(f, X_my_welch_CorrectMag, "o-", lw=3, label="my manual from stft")
plt.plot(Om, Xw, "o-", label=r"scipy's welch")
plt.ylim(0, 4)
plt.xlim(0, np.pi)
plt.xlabel(r"$\Omega$ / (rad/s)")
plt.ylabel(r"amplitude$^2$ / (rad/s)")
plt.title("PSD Welch Method, Hamming window, N$_\mathrm{DFT}=64$, $H_\mathrm{hop}=32$")
plt.legend()
plt.grid(True, which="both")

# Conclusion

Congratulations, we have done it :-)

To summarize:

- We have implemented the basic blocks for STFT-based analysis and synthesis using different overlap-add methods. It is comparably simple to add the further steps such that we can apply time-frequency processing for e.g. filtering.

- Then we checked in detail how a surface plot of STFT data is created. These insights and playing around with the parameters help to understand what is going on. We should get experience on our own regarding different time and different frequency resolution, zeropadding and impact of different windows. Once we set the basics here, these steps can be conveniently performed and better understood.

- At last, we got into detail of the modified periodogram, more specifically the Welch periodogram. This is (besides parametric approaches, see the lecture) one of fundamental 'need to know' in DSP, when dealing with random data, which in practice is most often the case. Here, we also should play around with different averages, different DFT block sizes, different windows to get experience on the made up signal...and then trying other signals.

Last reminder: After implementing stuff on our own, we might better understand the theoretical foundations and the equations of the discussed approaches. So, check back the lectures and text books, it will be enlightening.

Furthermore, we experienced that tiny details can make big differences. So, in-depth-checking things/tools, that we employ, is always a good idea when engineering.

# Literature on STFT Analysis / Synthesis Fundamentals

#### Articles
- M. Portnoff (1976): "Implementation of the digital phase vocoder using the fast Fourier transform." in: IEEE Transactions on Acoustics, Speech, and Signal Processing, vol. 24, no. 3, pp. 243-248, June 1976.
https://doi.org/10.1109/TASSP.1976.1162810

- J. Allen (1977): "Short term spectral analysis, synthesis, and modification by discrete Fourier transform." in: IEEE Transactions on Acoustics, Speech, and Signal Processing, vol. 25, no. 3, pp. 235-238, June 1977.
https://doi.org/10.1109/TASSP.1977.1162950

- J. B. Allen, L. R. Rabiner (1977): "A unified approach to short-time Fourier analysis and synthesis." in: Proceedings of the IEEE, vol. 65, no. 11, pp. 1558-1564, Nov. 1977.
https://doi.org/10.1109/PROC.1977.10770

- R. Crochiere (1980): "A weighted overlap-add method of short-time Fourier analysis/Synthesis." in: IEEE Transactions on Acoustics, Speech, and Signal Processing, vol. 28, no. 1, pp. 99-102, February 1980.
https://doi.org/10.1109/TASSP.1980.1163353

- M. Portnoff (1980): "Time-frequency representation of digital signals and systems based on short-time Fourier analysis." in IEEE Transactions on Acoustics, Speech, and Signal Processing, vol. 28, no. 1, pp. 55-69, February 1980.
https://doi.org/10.1109/TASSP.1980.1163359

- **[Gri84]** D. Griffin, Jae Lim (1984): "Signal estimation from modified short-time Fourier transform." in: IEEE Transactions on Acoustics, Speech, and Signal Processing, vol. 32, no. 2, pp. 236-243, April 1984.
https://doi.org/10.1109/TASSP.1984.1164317

- **[Har78]** F.J. Harris (1978): "On the Use of Windows for Harmonic Analysis with the Discrete Fourier Transform." in: Proceedings of the IEEE, vol. 66, no. 1, pp. 51-83, January 1978.
https://doi.org/10.1109/PROC.1978.10837

#### Books

- **[Rab78]** L.R. Rabiner, R.W. Schafer (1978): "Digital Processing of Speech Signals.", Prentice Hall, Englewood Cliffs, chapter 6.

- **[OS10]** A.V. Oppenheim, R.W. Schafer (2010): "Discrete-Time Signal Processing." Pearson, Upper Saddle River, 3rd ed. chapter 10.3.

- L.R. Rabiner, R.W. Schafer (2007): "Introduction to Digital Speech Processing, Foundations and Trends in Signal Processing, vol 1, no 1–2, pp 1–194, now Publishers, Delft, chapter 4. 
https://doi.org/10.1561/2000000001

- **[Zoe11]** U. Zölzer (2011): "DAFX: Digital Audio Effects." Wiley, Chichester, 2nd ed., chapter 7. https://onlinelibrary.wiley.com/doi/book/10.1002/9781119991298

- **[Rei15]** J.D. Reiss, A.P. McPherson (2015): "Audio Effects; Theory, Implementation and Application.", CRC Press, Boca Raton, 1st ed., chapter 8. https://doi.org/10.1201/b17593

- **[Pul18]** V. Pulkki, S. Delikaris-Manias, A. Politis (2018): "Parametric Time-Frequency Domain Spatial Audio." Wiley, Chichester, 1st ed., chapter 1.
https://onlinelibrary.wiley.com/doi/book/10.1002/9781119252634

#### Open Educational Resources

- **[Smi01]** Julius O. Smith III (2001): "Spectral Audio Signal Processing", Center for Computer Research in Music and Acoustics (CCRMA), Stanford University, https://ccrma.stanford.edu/~jos/sasp/sasp.html
